<a href="https://colab.research.google.com/github/filosoficaguilar/homicidios_sexenio_lopez_obrador/blob/master/Estadisticas_del_homicidios_periodo_2019_%3F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Toda la informacion presente es propiedad del gobierno de la republica mexicana, los datos son obtenidos directamente de http://www.informeseguridad.cns.gob.mx/ y procesados mediante python para la obtencion de datos estadisticos, el uso que se le de es responsabilidad de quien los use y no de quien lo esta programando.
Lo datos son crudos y nada mas se obtienen y analizan, no existe modificacion alguna.

In [0]:
! pip3 install tabula-py
! pip install pyPdf2

In [0]:
#IMPORTACION DE LIBRERIAS
import re
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
import webbrowser
import PyPDF2
from bs4 import BeautifulSoup #
from tabula import read_pdf 

In [0]:
def datos(): 
    
    url='http://www.informeseguridad.cns.gob.mx/'
    resp=requests.get(url) 
      
    cabeceras = ["Fecha", "Homicidios dolosos Fuentes abiertas"]
    hv1 = []
    cifrasTotales = []
    fechas = []
    
    if resp.status_code==200: 
        
        soup=BeautifulSoup(resp.text,'html.parser')     
        
        l=soup.find("tbody") 
        
        fullTable = l.findAll("a")
        
        for row in fullTable:
          if(".pdf" in row["href"] and "v2" not in row["href"]):
            if("homicidios" in row["href"]):
              hv1.append(row["href"])              
        
        hv1.reverse()
        
        for i in hv1:
          try:
            url = 'http://www.informeseguridad.cns.gob.mx/'+i
            dfs = read_pdf(url, pages = 'all',stream = True,output_format="json")
            dia = re.sub('[^0-9]', '', i)
            print(dia[0:2]+"/"+dia[2:4]+"/"+dia[4:])
            json.dumps(dfs)

            try:
              canDoIt = False
              for totales in dfs[-1]["data"][-1]:

                if("Total" in totales and canDoIt == False):
                  canDoIt = True
                if(re.match(r"[0-9]{2}\s{0,1}[0-9]{0,5}",totales["text"])):
                  dia = re.sub('[^0-9]', '', i)
                  fechas.append(dia[0:2]+"/"+dia[2:4]+"/"+dia[4:])
                  if(" " in totales["text"]):
                    val = totales["text"].split(" ")[0]
                    cifrasTotales.append(int(val))
                  else:
                    cifrasTotales.append(int(totales["text"]))
                  break
            except:
              continue
          except Exception as e: 
            print(e)
            
        plt.xlabel("Fechas")
        plt.xticks(rotation=90)
        plt.ylabel("Cantidad de asesinatos por dia")
        plt.title("Cantidad de asesinatos en periodo 2019-?")
        plt.scatter(fechas,cifrasTotales)
        plt.figure(figsize=(7.5,7.5))
        plt.show()

        dataset = pd.DataFrame(data = [fechas,cifrastotales], columns=cabeceras)
        print(dataset)


    else: 
        print("Error al conectar con la fuente")
    

datos()